In [1]:
import os
import glob
import sys

import numpy as np
import pandas as pd
from scipy.misc import derivative
import scipy.integrate as intg
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow.keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import Constant, RandomNormal

from sklearn.metrics import mean_squared_error
from astroNN.nn.layers import MCDropout

sys.path.append("/home/isidro/Documents/github/nnogada/")
from nnogada import Nnogada

np.random.seed(0)
%matplotlib inline

In [2]:
# absolute path to search all text files inside a specific folder
path = r'/home/isidro/Documents/github/model_independent_RC/data/MassModels/*.NFW.fix.REV.dat'
files = glob.glob(path)
print(files)

['/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC2903.NFW.fix.REV.dat', '/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC3521.NFW.fix.REV.dat', '/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC5055.NFW.fix.REV.dat', '/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC2841.NFW.fix.REV.dat', '/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC7331.NFW.fix.REV.dat', '/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC6946.NFW.fix.REV.dat', '/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC2366.NFW.fix.REV.dat', '/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC925.NFW.fix.REV.dat', '/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC3621.NFW.fix.REV.dat', '/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC2403.NFW.fix.REV.dat', '/home/isidro/Documents/github/model_independent_RC/data/Mas

In [3]:
for file in files:
    print(file)

len(files)

/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC2903.NFW.fix.REV.dat
/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC3521.NFW.fix.REV.dat
/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC5055.NFW.fix.REV.dat
/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC2841.NFW.fix.REV.dat
/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC7331.NFW.fix.REV.dat
/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC6946.NFW.fix.REV.dat
/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC2366.NFW.fix.REV.dat
/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC925.NFW.fix.REV.dat
/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC3621.NFW.fix.REV.dat
/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC2403.NFW.fix.REV.dat
/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC3031.NFW.fix.REV.dat


17

In [4]:
def model_regression_dropout(num_nodes, num_hidden_layers):
    # Defeine Keras model for regression
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(batch_input_shape=((None, 1))))
    for _ in range(num_hidden_layers):
        model.add(Dense(units=num_nodes, activation='relu'))
        model.add(MCDropout(0.2))
    model.add(Dense(units=2, activation="linear"))
    return model

In [5]:
files[0]

'/home/isidro/Documents/github/model_independent_RC/data/MassModels/NGC2903.NFW.fix.REV.dat'

In [ ]:
history_all = []

for idx, file in enumerate(files):
    print("Model {}/{}".format(idx+1, len(files)))
    print("-"*10)
    
    history_ind = {}
    
    data = np.loadtxt(files[idx], skiprows=12) 
    df = pd.DataFrame(data, columns=['Radius', 'vgas', 'vdisk', 'vbulge', 
                                     'vobs', 'err_vobs', 'Vu', 'Vt','Rxv', 'Vxy'])
    N = len(df.values)
    df = df[['Radius', 'vobs','err_vobs', 'Vt']]


    randomize = np.random.permutation(N)
    data = df.values[randomize]

    z = data[:,0]
    y = data[:,1:3]
    
    scalerz = StandardScaler()
    scalerz.fit(z.reshape(-1,1))
    # apply transform
    z = scalerz.transform(z.reshape(-1,1))
    
    split = 0.8
    ntrain = int(split * len(z))
    indx = [ntrain]
    z_train, z_test = np.split(z, indx)
    y_train, y_test = np.split(y, indx)
    
    print("X_train shape: {} | y_train shape: {} | x_val shape: {} | y_val shape: {}".format(np.shape(z_train), 
                                                                                             np.shape(y_train), 
                                                                                             np.shape(z_test), 
                                                                                             np.shape(y_test)))

    population_size = 6  # max of individuals per generation
    max_generations = 8    # number of generations
    gene_length = 8        # lenght of the gene, depends on how many hiperparameters are tested 2*hyp
    k = 1                  # num. of finalist individuals

    # Define the hyperparameters for the search
    hyperparams = {'deep': [3, 4], 'num_units': [50, 100, 150, 200], 'batch_size': [4, 8, 16, 32]}

    # generate a Nnogada instance
    net_fit = Nnogada(hyp_to_find=hyperparams, X_train=z_train, Y_train=y_train, X_val=z_test, Y_val=y_test, 
                      neural_library='keras', regression=True)
    # Set the possible values of hyperparameters and not use the default values from hyperparameters.py
    net_fit.set_hyperparameters()
          
    best_population = net_fit.ga_with_elitism(population_size, max_generations, gene_length, k)
          
    deep = int(best_population['deep'])
    num_units = int(best_population['num_units'])
    batch_size = int(best_population['batch_size'])
            
    optimizer = Adam(lr=0.0005)
    
    model = model_regression_dropout(num_hidden_layers=deep, num_nodes=num_units)

    # Compile Keras model
    model.compile(loss='mse', optimizer=optimizer)  
    model_train = model.fit(z_train, y_train, batch_size=batch_size,
                            epochs=1000,verbose=1,
                            validation_data=(z_test, y_test))
                     
    history_ind['idx'] = idx
    history_ind['deep'] = deep
    history_ind['batch_size'] = batch_size
    history_ind['num_units'] = num_units
    history_ind['val_loss'] = model_train.history['val_loss'][-1]
          
    history_all.append(history_ind)   
         
    model.save('models/RC_model_{}.h5'.format(idx+1))
  
    # Generate test data
    test_batch_size = 500
    # x_test = np.random.uniform(0, 2., test_batch_size)
    x_test = np.linspace(min(df['Radius'].values)-0.1, max(df['Radius'].values)+0.1, test_batch_size)

    mc_dropout_num = 100  # Run Dropout 100 times
    predictions = np.zeros((mc_dropout_num, test_batch_size, 2))
    for i in range(mc_dropout_num):
        predictions[i] = model.predict(scalerz.transform(x_test.reshape(-1,1)))

    # get mean results and its varience
    prediction_mc_dropout = np.mean(predictions, axis=0)
    std_mc_dropout = np.std(predictions, axis=0)
    
    # plt.rcParams["figure.figsize"] = [7.50, 3.50]
    plt.rcParams["figure.autolayout"] = True
    overlapping = 0.6
    # Plotting
    plt.figure(figsize=(10, 7), dpi=100)
    sigma = np.sqrt(std_mc_dropout[:, 0]**2 + std_mc_dropout[:, 1]**2+ prediction_mc_dropout[:,1]**2)

    plt.plot(df['Radius'].values, df['Vt'].values, color='k', linewidth=3, label='NFW theory',alpha=1)

    plt.errorbar(df['Radius'].values, df['vobs'].values, yerr=df['err_vobs'].values, fmt='.', 
                 color='red', elinewidth=1, ecolor='red', markersize=5, label='Observations', alpha=overlapping)

    plt.errorbar(x_test, prediction_mc_dropout[:,0], yerr=sigma, markersize=2, fmt='o', 
                 ecolor='green', capthick=2, elinewidth=0.5, alpha=overlapping-0.2, c='green',
                 label='Neural reconstruction')


    plt.ylabel("$V(r)$", fontsize=20)
    plt.xlabel("r", fontsize=20)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)

    plt.legend(loc='upper left')
    plt.tight_layout()

    plt.savefig("neural_reconstruction_RC_{}.png".format(idx+1), dpi=100)
          
    plt.figure(figsize=(8, 6), dpi=100)

    plt.plot(model_train.history['loss'], color='r', )
    plt.plot(model_train.history['val_loss'], color='g')

    plt.ylabel('MSE', fontsize=11)
    plt.xlabel('Epoch', fontsize=11)
    plt.legend(['training set', 'validation set'], loc='upper right', fontsize=12)
    plt.xticks(fontsize=10)
    plt.savefig('loss_fn_neural_RC_{}'.format(idx+1), dpi=100)
    print("-"*10)
    # plt.ylim(0, 200)



  0%|          | 0/8 [00:00<?, ?it/s]

Model 1/17
----------
X_train shape: (69, 1) | y_train shape: (69, 2) | x_val shape: (18, 1) | y_val shape: (18, 2)
Using CPU


100%|██████████| 8/8 [06:08<00:00, 45.52s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.



Best 5 solutions:
-----------------

   deep  num_units  batch_size          loss         score       time
0     4        150           8  19180.539062  19180.539062  11.434159
1     4        150           8  19180.539062  19180.539062  10.707484
2     4        200           8  19180.539062  19180.539062  11.724317
3     4        200          16  19180.539062  19180.539062   9.314642
4     4        150           8  19180.539062  19180.539062  12.007706
Epoch 1/1000
9/9 [==============================] - 2s 57ms/step - loss: 19334.0195 - val_loss: 19236.3965
Epoch 2/1000
9/9 [==============================] - 0s 14ms/step - loss: 19021.0625 - val_loss: 18705.3887
Epoch 3/1000
9/9 [==============================] - 0s 14ms/step - loss: 17892.7988 - val_loss: 16838.6250
Epoch 4/1000
9/9 [==============================] - 0s 11ms/step - loss: 14326.9717 - val_loss: 11215.5107
Epoch 5/1000
9/9 [==============================] - 0s 13ms/step - loss: 6730.2705 - val_loss: 2868.9795
Epoch 6/1

9/9 [==============================] - 0s 17ms/step - loss: 267.4521 - val_loss: 238.0033
Epoch 76/1000
9/9 [==============================] - 0s 14ms/step - loss: 262.7821 - val_loss: 254.1159
Epoch 77/1000
9/9 [==============================] - 0s 14ms/step - loss: 239.5647 - val_loss: 285.5418
Epoch 78/1000
9/9 [==============================] - 0s 11ms/step - loss: 291.4614 - val_loss: 258.6479
Epoch 79/1000
9/9 [==============================] - 0s 13ms/step - loss: 234.7593 - val_loss: 297.2886
Epoch 80/1000
9/9 [==============================] - 0s 14ms/step - loss: 276.4422 - val_loss: 226.1358
Epoch 81/1000
9/9 [==============================] - 0s 13ms/step - loss: 204.8457 - val_loss: 238.2350
Epoch 82/1000
9/9 [==============================] - 0s 14ms/step - loss: 269.2858 - val_loss: 219.8131
Epoch 83/1000
9/9 [==============================] - 0s 14ms/step - loss: 218.0018 - val_loss: 233.0119
Epoch 84/1000
9/9 [==============================] - 0s 14ms/step - loss: 230.

...

4/4 [==============================] - 0s 15ms/step - loss: 6.4267 - val_loss: 4.7263
Epoch 956/1000
4/4 [==============================] - 0s 15ms/step - loss: 10.9120 - val_loss: 13.8626
Epoch 957/1000
4/4 [==============================] - 0s 15ms/step - loss: 11.7426 - val_loss: 11.6625
Epoch 958/1000
4/4 [==============================] - 0s 15ms/step - loss: 12.1662 - val_loss: 10.4054
Epoch 959/1000
4/4 [==============================] - 0s 15ms/step - loss: 8.5293 - val_loss: 7.8146
Epoch 960/1000
4/4 [==============================] - 0s 16ms/step - loss: 7.7355 - val_loss: 6.1525
Epoch 961/1000
4/4 [==============================] - 0s 15ms/step - loss: 10.7163 - val_loss: 11.5016
Epoch 962/1000
4/4 [==============================] - 0s 14ms/step - loss: 17.9186 - val_loss: 6.6640
Epoch 963/1000
4/4 [==============================] - 0s 14ms/step - loss: 15.1827 - val_loss: 10.0799
Epoch 964/1000
4/4 [==============================] - 0s 14ms/step - loss: 11.0264 - val_loss: 

16/16 [==============================] - 0s 2ms/step


/home/isidro/.local/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/isidro/.local/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
  0%|          | 0/8 [00:00<?, ?it/s]

----------
Model 8/17
----------
X_train shape: (76, 1) | y_train shape: (76, 2) | x_val shape: (20, 1) | y_val shape: (20, 2)
Using CPU


 50%|█████     | 4/8 [01:58<01:52, 28.06s/it]